# Workbook for optimizing foci segmentation thresholds

This workbook provides step-by-step instructions for determining segmentation thresholds for identifying foci within cells. __Do not edit this workbook file directly!__ Make a copy for each optimization that you perform and edit that version. Save this template in an unedited form for future experiments.

This workbook uses a built-in "optimization mode" that only pulls the first three images out of multi-image .czi files to dramatically speed up processing. This will allow you to tinker with thresholds and find the ideal cutoffs.

I found that the most effective way to define these cutoffs was to use a positive control image - one where there were many foci - and a negative control image with few to no foci to set the thresholds. This allows you to identify cutoffs that simultaneously maximize sensitivity while minimizing false positives.

__Before running this notebook, you must have completed the setup notebook in the install folder.__

In [2]:
# import required code
import sys
sys.path.append('/Users/nweir/Dropbox/code/csth-imaging')
from csth_analysis import find_cells, segment_cells, foci

/usr/local/lib/python3.6/site-packages/tifffile.py:313: UserWarning: ImportError: No module named '_tifffile'. Loading of some compressed images will be very slow. Tifffile.c can be obtained at http://www.lfd.uci.edu/~gohlke/
  "ImportError: No module named '_tifffile'. "
/usr/local/lib/python3.6/site-packages/czifile.py:130: UserWarning: failed to import the optional _czifile C extension module.
Decoding of JXR and JPEG encoded images will not be available.
Czifile.pyx can be obtained at http://www.lfd.uci.edu/~gohlke/
  "failed to import the optional _czifile C extension module.\n"
/usr/local/lib/python3.6/site-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


The cell below defines the file paths for the files you will need during this process. Change each variable as needed.

In [3]:
neg_ctrl_czi = '/Users/nweir/Dropbox/chris_imaging/imaging_test/th_sample_czi/lc3_p62_timecourse/LC3-p62_HEK_dFIP200_NoTreat_10Pos_1_AiryscanProcessing.czi'
pos_ctrl_czi = '/Users/nweir/Dropbox/chris_imaging/imaging_test/th_sample_czi/lc3_p62_timecourse/LC3-p62_HEK_dTMEM_1hrTor_10Pos_1_AiryscanProcessing.czi'
neg_empty_field_czi = '/Users/nweir/Dropbox/chris_imaging/imaging_test/th_sample_czi/lc3_p62_timecourse/LC3-p62_HEK_dFIP200_NoTreat_Empty_AiryscanProcessing.czi'
pos_empty_field_czi = '/Users/nweir/Dropbox/chris_imaging/imaging_test/th_sample_czi/lc3_p62_timecourse/LC3-p62_HEK_dTMEM_1hrTor_Empty_AiryscanProcessing.czi'
svm_pkl = '/Users/nweir/Dropbox/code/csth-imaging/trained_svm.pkl' # change /path/to/ so it points to the csth-imaging folder

Now that you have defined the important variables, you will perform pre-processing. This takes the following steps:
1. Load images
2. Identify regions of the field containing cells
3. Identify nuclei
4. Segment cells  
    _Note:_ By default this segments cells using the 488 wavelength. Change 488 to your desired wavelength in the splitter.segment_cells() command to change wavelengths.
5. Initialize foci detection object.

The steps are indicated in the code blocks.

In [4]:
## NOTE: RUNNING

# 1. load images and the trained SVM focus classifier, and 2.
neg_ctrl_finder = find_cells.MultiFinder(filename=neg_ctrl_czi,
                                         bg_filename=neg_empty_field_czi,
                                         oof_svm=svm_pkl,
                                         optim=True)
pos_ctrl_finder = find_cells.MultiFinder(filename=pos_ctrl_czi,
                                         bg_filename=pos_empty_field_czi,
                                         oof_svm=svm_pkl,
                                         optim=True)
print('MultiFinder created.')

# 2. find cells
neg_ctrl_splitter = segment_cells.CellSplitter(neg_ctrl_finder)
pos_ctrl_splitter = segment_cells.CellSplitter(pos_ctrl_finder)

# 3. identify nuclei
neg_ctrl_splitter.segment_nuclei(verbose=True)
pos_ctrl_splitter.segment_nuclei(verbose=True)

# 4. Segment cells.
neg_ctrl_splitter.segment_cells(488, verbose=True)
pos_ctrl_splitter.segment_cells(488, verbose=True)

# 5. Initialize foci detection objects.
neg_foci = foci.Foci(neg_ctrl_splitter, verbose=True)
pos_foci = foci.Foci(pos_ctrl_splitter, verbose=True)

Optimization mode: only using first three images.
Optimization mode: only using first three images.
MultiFinder created.
generating cell masks...
log-transforming arrays...
applying gaussian filter...
computing p-value transformation...
 computing p-val xform for image 1 out of 3
 computing p-val xform for image 2 out of 3
 computing p-val xform for image 3 out of 3
converting to binary...

generating cell masks...

generating mask #1
labeling contiguous objects...
eliminating objects w/volume < 100,000 px...
pruning labels...
unlabeling out of focus slices...
generating gradient image...
generating gradient intensity histograms...
predicting focus labels using SVM classifier...
calculating decision function values for labels...
raw focus labels:
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
correcting slice labels...
One continuous stretch defined as in focus.
appending outputs...
mask #1 complete.

generating mask #2
labeling

raw image imported.
preprocessing complete.
filling holes in objects.
holes filled.
distance map complete.
smoothing distance map...
distance map smoothed.
identifying maxima...
maxima identified.
watershedding...
watershedding complete.
creating PexSegmentObj...
passing segmented objs and seeds to Nuclei instance...

segmenting image #2 of 3...
Using thresholds set based on slice intensity
thresholding floor set to 3509
slice-by-slice cutoffs:
[  259.5   249.    250.5   267.    289.5   353.5   503.5   722.5  1189.
  2006.   3168.   4563.   5833.   6926.5  7882.   8398.   8487.5  8446.
  8639.5  8520.5  8548.5  8744.5  8774.   8581.   8340.5  8046.5  7470.5
  6646.5  5539.5  4424.   3548.   2832.5  2231.5  1700.5  1310.   1019.
   839.5   724.    575.    499.    392.    324.5   295.5   307.5   278.
   277.5   254.5   203.5   195.    176.5   168. ]
generating threshold image...
raw image imported.
preprocessing complete.
filling holes in objects.
holes filled.
distance map complete.
smo

/usr/local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


cell mask means:
[1732.8113001192426, 1793.4162702511028, 1855.3783902627229, 1915.9120247062222, 1972.4163322030129, 2023.6643205757302, 2069.5430382819991, 2110.5859807727761, 2147.595701905092, 2181.0876722561434]
slopes:
[61.28354507174015, 61.247877227559684, 58.518970970145006, 53.876147934754044, 48.563353039493109, 43.460830098522933, 39.02633181154647, 35.250845741683634]
slope deltas:
[ 0.03566784  2.72890626  4.64282304  5.3127949   5.10252294  4.43449829
  3.77548607]
desired erosions: 5
erosion of mask #0 complete.
eroding mask 1...
cell mask means:
[1448.4206456496045, 1502.6095819370344, 1557.1959634454324, 1609.4374516874741, 1657.1693481299933, 1699.9087619348409, 1738.0074961519108, 1772.4452368759273, 1804.0915214496536, 1833.4216545951406]
slopes:
[54.38765889791398, 53.413934875219866, 49.986692342280435, 45.235655123683387, 40.419074010958752, 36.268237470543227, 33.042012648871378, 30.48820885960663]
slope deltas:
[ 0.97372402  3.42724253  4.75103722  4.81658111 

Next, you're ready to try segmenting foci and checking the output to see if it looks correct.  
A bit of background on how Canny edge detection works so you can logically select values. There are two threshold numbers that go into detecting foci:
- high threshold: the minimum edge intensity to _seed_ a new object. The entire object doesn't have to have an edge this sharp, but if no part of it does, it won't be identified.
- low threshold: the minimum edge intensity to _grow_ an object. If one point already satisfied the high threshold, it can grow even if it has a dimmer edge as long as that dimmer edge is above the low threshold.  

If this doesn't make sense, read about the Canny edge detection algorithm on Wikipedia or elsewhere.

I often start with values of 10,000 and 7,500 for the high and low thresholds respectively, so those are the default values below.

Running the code block below will perform segmentation and save the raw images as well as the segmented foci to the output_dir you select in the first line. You can then open these images in fiji, overlay the segmented foci onto the raw images, and determine if the threshold is too high/too low. The code will also output some descriptive information about the number of dots, their intensities, etc. which may be useful to you in deciding thresholds.

In [5]:
from skimage import io  # for saving images
output_dir = '/Users/nweir/Dropbox/chris_imaging/timelapse_test/'  # change this to where you'd like the output images to go.
import os
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)
# save raw images
for c in pos_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, pos_foci.imgs[c].shape[0]):  # for each image in that channel
        io.imsave(output_dir + 'pos_raw_' + str(c) + '_' + str(im) + '.tif',
                 pos_foci.imgs[c][im, :, :, :])
for c in neg_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, neg_foci.imgs[c].shape[0]):  # for each image in that channel
        io.imsave(output_dir + 'neg_raw_' + str(c) + '_' + str(im) + '.tif',
                 neg_foci.imgs[c][im, :, :, :])        

/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/timelapse_test/pos_raw_488_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/timelapse_test/pos_raw_561_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/timelapse_test/pos_raw_561_1.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/timelapse_test/pos_raw_561_2.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/timelapse_test/neg_raw_488_1.tif is a low 

In [14]:
## RUNNING THIS BLOCK WILL TAKE A WHILE - BE PATIENT! ##
# edit the (high, low) threshold pairs below to optimize segmentation.
green_thresholds = (14000, 10500) 
red_thresholds = (20000, 15000)
# perform segmentation
neg_foci.segment(verbose=True, thresholds={488: green_thresholds,
                                          561: red_thresholds})
pos_foci.segment(verbose=True, thresholds={488: green_thresholds,
                                          561: red_thresholds})
# save segmentation outputs
for c in pos_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, len(pos_foci.foci[c])):  # for each image in that channel
        io.imsave(output_dir + 'pos_foci_' + str(c) + '_' + str(im) + '.tif',
                 pos_foci.foci[c][im])
for c in neg_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, len(neg_foci.foci[c])):  # for each image in that channel
        io.imsave(output_dir + 'neg_foci_' + str(c) + '_' + str(im) + '.tif',
                 neg_foci.foci[c][im])

beginning segmentation.
------------------------------------------------------
segmenting foci from channel 488
------------------------------------------------------
canny threshold for channel 488: (13500, 10000)
segmenting foci for position 1 out of 3
generating normalized image for segmentation...
blurring image...
calculating image mean...
original cell mean: 2011.78324413
normalizing image...
performing segmentation...
raw image imported.
performing gaussian filtering...
Image smoothed.
preprocessing complete.
generating distance map...
distance map complete.
smoothing distance map...
distance map smoothed.
identifying maxima...
maxima identified.
watershedding...
watershedding complete.
creating PexSegmentObj...
eliminating dim foci...
before eliminating dim foci: 20 foci in image
cell mean: 990.691041753
cell standard deviation: 1220.36429739
intensity cutoff for removal: 6482.33038001
-----filtering foci-----
focus intensity: 105.471015694
removing focus
focus intensity: 8045.

------------------------------------------------------
beginning segmentation.
------------------------------------------------------
segmenting foci from channel 488
------------------------------------------------------
canny threshold for channel 488: (13500, 10000)
segmenting foci for position 1 out of 3
generating normalized image for segmentation...
blurring image...
calculating image mean...
original cell mean: 1108.18269446
normalizing image...
performing segmentation...
raw image imported.
performing gaussian filtering...
Image smoothed.
preprocessing complete.
generating distance map...
distance map complete.
smoothing distance map...
distance map smoothed.
identifying maxima...
maxima identified.
watershedding...
watershedding complete.
creating PexSegmentObj...
eliminating dim foci...
before eliminating dim foci: 162 foci in image
cell mean: 1010.02572825
cell standard deviation: 1144.46402784
intensity cutoff for removal: 6160.11385353
-----filtering foci-----
focus intens

focus intensity: 7921.63636364
focus intensity: 8563.23853211
focus intensity: 8041.2755102
focus intensity: 9479.28571429
focus intensity: 5666.8125
removing focus
focus intensity: 8280.05960265
focus intensity: 8079.48421053
focus intensity: 12518.9357798
focus intensity: 13240.0157895
focus intensity: 8234.84920635
focus intensity: 8705.88181818
focus intensity: 8268.84210526
focus intensity: 7119.36363636
focus intensity: 8030.69230769
focus intensity: 11831.5294118
focus intensity: 12681.6125
focus intensity: 11719.4833333
focus intensity: 13737.2
focus intensity: 11311.6046512
focus intensity: 8023.2
focus intensity: 7687.16393443
focus intensity: 9120.17117117
focus intensity: 7818.46
focus intensity: 9126.22916667
focus intensity: 5573.975
removing focus
focus intensity: 6587.54545455
removing focus
focus intensity: 13022.1818182
focus intensity: 8674.57317073
focus intensity: 10919.0634921
focus intensity: 11511.2727273
focus intensity: 13475.3247423
focus intensity: 8774.9316

focus intensity: 6938.125
removing focus
focus intensity: 7819.875
focus intensity: 10890.8461538
focus intensity: 9941.32307692
focus intensity: 6149.77464789
removing focus
focus intensity: 8405.84210526
focus intensity: 13661.752809
focus intensity: 5388.30952381
removing focus
focus intensity: 6680.73770492
removing focus
focus intensity: 9348.85185185
focus intensity: 7016.75438596
removing focus
focus intensity: 8455.23809524
focus intensity: 7035.84507042
removing focus
focus intensity: 11483.4657534
focus intensity: 11823.3699187
focus intensity: 7623.88059701
focus intensity: 12206.8021978
focus intensity: 9707.41610738
focus intensity: 7261.53424658
focus intensity: 6042.86363636
removing focus
focus intensity: 8771.36885246
focus intensity: 14276.7073171
focus intensity: 7509.30188679
focus intensity: 5517.81481481
removing focus
focus intensity: 10385.3412698
focus intensity: 7343.55555556
focus intensity: 6576.25742574
removing focus
focus intensity: 7151.6
focus intensity

focus intensity: 11440.953125
focus intensity: 12534.5138889
focus intensity: 7432.48148148
focus intensity: 6143.76190476
removing focus
focus intensity: 6573.41666667
removing focus
focus intensity: 8242.52866242
focus intensity: 10192.1567164
focus intensity: 11352.2601626
focus intensity: 9219.38961039
focus intensity: 7570.4
focus intensity: 10268.5851852
focus intensity: 9745.0619469
focus intensity: 8597.21367521
focus intensity: 11453.3045977
focus intensity: 6795.2962963
removing focus
focus intensity: 7146.59574468
focus intensity: 7923.0
focus intensity: 8874.92727273
focus intensity: 6804.75
removing focus
focus intensity: 7226.03508772
focus intensity: 7900.58536585
focus intensity: 11832.509434
focus intensity: 9977.82285714
focus intensity: 11361.6965517
focus intensity: 8790.69491525
focus intensity: 7900.43298969
focus intensity: 6104.64788732
removing focus
focus intensity: 11139.7346939
focus intensity: 7766.27777778
focus intensity: 7710.57692308
focus intensity: 71

focus intensity: 13444.0606061
focus intensity: 9355.65217391
focus intensity: 14645.3863636
focus intensity: 9367.95652174
focus intensity: 12128.3888889
focus intensity: 11791.2597403
focus intensity: 11297.7735849
focus intensity: 15073.4884793
focus intensity: 12030.2352941
focus intensity: 15963.0490196
focus intensity: 16338.3454545
focus intensity: 30163.3606557
focus intensity: 14604.2222222
focus intensity: 9671.06944444
focus intensity: 11285.4606742
focus intensity: 25767.2064677
focus intensity: 8596.77777778
focus intensity: 8700.44444444
focus intensity: 9782.25333333
focus intensity: 10685.4285714
focus intensity: 12738.3382353
focus intensity: 12360.6593407
focus intensity: 18216.7722222
focus intensity: 13337.3555556
focus intensity: 9965.93478261
focus intensity: 17428.9120879
focus intensity: 19642.1596639
focus intensity: 13454.1717172
focus intensity: 10361.8684211
focus intensity: 12384.0326087
focus intensity: 12248.283871
focus intensity: 10029.359375
focus inte

focus intensity: 23664.6233766
focus intensity: 19985.4008811
focus intensity: 34197.0878049
focus intensity: 16990.1917808
focus intensity: 18377.0751445
focus intensity: 11253.4927536
focus intensity: 32832.9056604
focus intensity: 17052.2574257
focus intensity: 13427.4205607
focus intensity: 15708.1685393
focus intensity: 8304.66666667
removing focus
focus intensity: 16505.0373832
focus intensity: 23742.804878
focus intensity: 30596.9810127
focus intensity: 17928.3962264
focus intensity: 15478.1449275
focus intensity: 27662.554007
focus intensity: 30127.3539095
focus intensity: 23142.9913793
focus intensity: 13737.125
focus intensity: 17356.0647059
focus intensity: 9729.29032258
focus intensity: 21141.0067568
focus intensity: 33179.5910931
focus intensity: 21687.5393939
focus intensity: 15098.39
focus intensity: 11133.2727273
focus intensity: 12286.96875
focus intensity: 10786.1538462
focus intensity: 25321.4579439
focus intensity: 9400.5443038
focus intensity: 14450.8981481
focus i

removing focus
focus intensity: 17246.2016807
focus intensity: 21244.9153439
focus intensity: 9518.3125
focus intensity: 19528.9195804
focus intensity: 22355.7183673
focus intensity: 12892.5041322
focus intensity: 20962.8878924
focus intensity: 21169.5787037
focus intensity: 11030.82
focus intensity: 18955.0327869
focus intensity: 18477.0522388
focus intensity: 13642.3333333
focus intensity: 16515.6646884
focus intensity: 18008.3982301
focus intensity: 24608.5957447
focus intensity: 12586.1355932
focus intensity: 10946.369863
focus intensity: 14877.2900763
focus intensity: 16479.8586387
focus intensity: 15807.1363636
focus intensity: 13312.1428571
focus intensity: 16880.3306452
focus intensity: 13630.4196429
focus intensity: 9946.27380952
focus intensity: 16806.9375
focus intensity: 15980.9007634
focus intensity: 18846.615942
focus intensity: 16240.4557823
focus intensity: 16088.2772277
focus intensity: 17379.2876712
focus intensity: 9479.62222222
focus intensity: 14914.2527473
focus i

after eliminating dim foci: 376 foci in image
eliminating foci that reside outside of cells...
eliminating intranuclear foci...
376 final foci
foci segmented from position 3

------------------------------------------------------
segmentation complete for channel 561
------------------------------------------------------


/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/timelapse_test/pos_foci_488_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/timelapse_test/pos_foci_488_1.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/timelapse_test/pos_foci_488_2.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/timelapse_test/pos_foci_561_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/timelapse_test/pos_foci_561_1.tif is a

In [12]:
for c in pos_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, len(pos_foci.foci[c])):  # for each image in that channel
        io.imsave(output_dir + 'pos_foci_' + str(c) + '_' + str(im) + '.tif',
                 pos_foci.foci[c][im])
for c in neg_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, len(neg_foci.foci[c])):  # for each image in that channel
        io.imsave(output_dir + 'neg_foci_' + str(c) + '_' + str(im) + '.tif',
                 neg_foci.foci[c][im])

/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/timelapse_test/pos_foci_488_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/timelapse_test/pos_foci_488_1.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/timelapse_test/pos_foci_488_2.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/timelapse_test/pos_foci_561_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/timelapse_test/pos_foci_561_1.tif is a

Edit the thresholds in the cell above and re-run as needed until you reach ideal thresholds.